In [1]:
import numpy as np
import pandas as pd

from datetime import date

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [84]:
train_offline_data_type = {'User_id': np.int64, 'Merchant_id': np.int64, 'Coupon_id': object,
              'Discount_rate': object, 'Distance' :object, 'Date_received': object,
              'Date': object}
test_offline_data_type = {'User_id': np.int64, 'Merchant_id': np.int64, 'Coupon_id': object,
              'Discount_rate': object, 'Distance' :object, 'Date_received': object}
train_online_data_type = {'User_id': np.int64, 'Merchant_id': np.int64, 'Action': np.int64,
              'Coupon_id': object, 'Discount_rate': object, 'Distance' :object,
              'Date_received': object}
              
test_offline = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/ccf_offline_stage1_test_revised.csv', dtype=test_offline_data_type)
train_offline = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/ccf_offline_stage1_train.csv', dtype=train_offline_data_type)
train_online = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/ccf_online_stage1_train.csv', dtype=train_online_data_type)
submission = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/sample_submission.csv')

In [85]:
print('test_offline dataset has {} rows and {} columns.'.format(test_offline.shape[0],test_offline.shape[1]))
print('train_offline dataset has {} rows and {} columns.'.format(train_offline.shape[0],train_offline.shape[1]))
print('train_online dataset has {} rows and {} columns.'.format(train_online.shape[0],train_online.shape[1]))
print('submission dataset has {} rows and {} columns.'.format(submission.shape[0],submission.shape[1]))

test_offline dataset has 113640 rows and 6 columns.
train_offline dataset has 1754884 rows and 7 columns.
train_online dataset has 11429826 rows and 7 columns.
submission dataset has 13 rows and 4 columns.


# 数据批注

In [86]:
def label(row):
    if row['Date_received'] is np.nan:
        return -1
    if row['Date'] is not np.nan:
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0
train_offline['label'] = train_offline.apply(label, axis = 1)

## 转换时间

In [87]:
train_offline['Date'] = pd.to_datetime(train_offline['Date'], format='%Y%m%d')
train_offline['Date_received'] = pd.to_datetime(train_offline['Date_received'], format='%Y%m%d')
test_offline['Date_received'] = pd.to_datetime(test_offline['Date_received'], format='%Y%m%d')

In [88]:
def convertRate(row):
    """Convert discount to rate"""
    if row is np.nan:
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

In [89]:
train_offline['discount'] = train_offline['Discount_rate'].apply(convertRate)
test_offline['discount'] = test_offline['Discount_rate'].apply(convertRate)

train_offline['Distance'] = train_offline['Distance'].fillna(-1)
train_offline['Distance'] = train_offline['Distance'].astype(int)
train_offline.loc[train_offline['Distance']==-1,'Distance'] = np.nan
test_offline['Distance'] = test_offline['Distance'].fillna(-1)
test_offline['Distance'] = test_offline['Distance'].astype(int)
test_offline.loc[test_offline['Distance']==-1,'Distance'] = np.nan

In [96]:
train = train_offline.copy()
test = test_offline.copy()

In [91]:
def get_offline_features(X, offline):
    # X = X[:1000]

    print(len(X), len(X.columns))

    coupon_id_notnull = offline[offline.Coupon_id.notnull()]

    temp = coupon_id_notnull
    coupon_consume = temp[temp.Date.notnull()]
    coupon_no_consume = temp[temp.Date.isnull()]

    user_coupon_consume = coupon_consume.groupby('User_id')

    X['weekday'] = X.Date_received.dt.weekday
    X['day'] = X.Date_received.dt.day

    # # 距离优惠券消费次数
    # temp = coupon_consume.groupby('Distance').size().reset_index(name='distance_0')
    # X = pd.merge(X, temp, how='left', on='Distance')
    #
    # # 距离优惠券不消费次数
    # temp = coupon_no_consume.groupby('Distance').size().reset_index(name='distance_1')
    # X = pd.merge(X, temp, how='left', on='Distance')
    #
    # # 距离优惠券领取次数
    # X['distance_2'] = X.distance_0 + X.distance_1
    #
    # # 距离优惠券消费率
    # X['distance_3'] = X.distance_0 / X.distance_2

    # temp = coupon_consume[coupon_consume.Distance != 11].groupby('Distance').size()
    # temp['d4'] = temp.Distance.sum() / len(temp)
    # X = pd.merge(X, temp, how='left', on='Distance')

    '''user features'''

    # 用户优惠券消费次数
    temp = user_coupon_consume.size().reset_index(name='u2')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户优惠券不消费次数
    temp = coupon_no_consume.groupby('User_id').size().reset_index(name='u3')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户优惠券消费与不消费次数比值
    X['u19'] = X.u2 / X.u3

    # 用户领取优惠券次数
    X['u1'] = X.u2.fillna(0) + X.u3.fillna(0)

    # 用户优惠券使用率
    X['u4'] = X.u2 / X.u1

    # 用户普通消费次数
    date_received_isnull_date_notnull = offline[offline.Date_received.isnull() & offline.Date.notnull()]

    temp = date_received_isnull_date_notnull.groupby('User_id').size().reset_index(name='u5')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户优惠券消费和普通消费次数
    X['u25'] = X.u2 + X.u5

    # 用户使用优惠券消费占比
    X['u20'] = X.u2 / X.u25

    # 用户所有消费平均间隔
    temp = offline[offline.Date.notnull()]
    temp = pd.merge(temp, temp.groupby('User_id').Date.max().reset_index(name='_max'))
    temp = pd.merge(temp, temp.groupby('User_id').Date.min().reset_index(name='_min'))
    temp = pd.merge(temp, temp.groupby('User_id').size().reset_index(name='_len'))
    temp['u6'] = (temp._max - temp._min).dt.days / (temp._len - 1)
    temp.drop_duplicates('User_id', inplace=True)
    X = pd.merge(X, temp[['User_id', 'u6']], how='left', on='User_id')

    # 用户普通消费平均间隔
    temp = date_received_isnull_date_notnull
    temp = pd.merge(temp, temp.groupby('User_id').Date.max().reset_index(name='_max'))
    temp = pd.merge(temp, temp.groupby('User_id').Date.min().reset_index(name='_min'))
    temp = pd.merge(temp, temp.groupby('User_id').size().reset_index(name='_len'))
    temp['u26'] = (temp._max - temp._min).dt.days / (temp._len - 1)
    temp.drop_duplicates('User_id', inplace=True)
    X = pd.merge(X, temp[['User_id', 'u26']], how='left', on='User_id')

    # 用户优惠券消费平均间隔
    temp = pd.merge(coupon_consume, user_coupon_consume.Date.max().reset_index(name='_max'))
    temp = pd.merge(temp, temp.groupby('User_id').Date.min().reset_index(name='_min'))
    temp = pd.merge(temp, temp.groupby('User_id').size().reset_index(name='_len'))
    temp['u7'] = (temp._max - temp._min).dt.days / (temp._len - 1)
    temp.drop_duplicates('User_id', inplace=True)
    X = pd.merge(X, temp[['User_id', 'u7']], how='left', on='User_id')

    # 用户15天内平均会普通消费几次
    X['u8'] = X.u6 / 15

    # 用户15天内平均会优惠券消费几次
    X['u9'] = X.u7 / 15

    # 用户优惠券领取到使用平均间隔
    temp = coupon_consume.copy()
    temp['days'] = (temp.Date - temp.Date_received).dt.days
    temp = (temp.groupby('User_id').days.sum() / temp.groupby('User_id').size()).reset_index(name='u10')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户在15天内使用掉优惠券的值大小
    X['u11'] = X.u10 / 15

    # 领取优惠券到使用优惠券间隔小于15天的次数
    temp = coupon_consume.copy()
    temp['days'] = (temp.Date - temp.Date_received).dt.days
    temp = temp[temp.days <= 15]
    temp = temp.groupby('User_id').size().reset_index(name='u21')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户15天使用掉优惠券的次数除以使用优惠券的次数
    X['u22'] = X.u21 / X.u2

    # 用户15天使用掉优惠券的次数除以领取优惠券未消费的次数
    X['u23'] = X.u21 / X.u3

    # 用户15天使用掉优惠券的次数除以领取优惠券的总次数
    X['u24'] = X.u21 / X.u1

    # 用户核销优惠券的平均折率
    temp = user_coupon_consume.discount.mean().reset_index(name='u45')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户核销优惠券的最低折率
    temp = user_coupon_consume.discount.min().reset_index(name='u27')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户核销优惠券的最高消费折率
    temp = user_coupon_consume.discount.max().reset_index(name='u28')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户核销过的不同优惠券数量
    temp = coupon_consume.groupby(['User_id', 'Coupon_id']).size()
    temp = temp.groupby('User_id').size().reset_index(name='u32')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户领取所有不同优惠券数量
    date_received_notnull = offline[offline.Date_received.notnull()]
    temp = date_received_notnull
    temp = temp.groupby(['User_id', 'Coupon_id']).size().reset_index(name='u47')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Coupon_id'])

    # 用户核销过的不同优惠券数量占所有不同优惠券的比重
    X['u33'] = X.u32 / X.u47

    # 用户平均每种优惠券核销多少张
    X['u34'] = X.u2 / X.u47

    # 核销优惠券用户-商家平均距离
    temp = offline[offline.Coupon_id.notnull() & offline.Date.notnull() & offline.Distance.notnull()]
    temp = temp.groupby('User_id').Distance
    temp = pd.merge(temp.count().reset_index(name='x'), temp.sum().reset_index(name='y'), on='User_id')
    temp['u35'] = temp.y / temp.x
    temp = temp[['User_id', 'u35']]
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户核销优惠券中的最小用户-商家距离
    temp = coupon_consume[coupon_consume.Distance.notnull()]
    temp = temp.groupby('User_id').Distance.min().reset_index(name='u36')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户核销优惠券中的最大用户-商家距离
    temp = coupon_consume[coupon_consume.Distance.notnull()]
    temp = temp.groupby('User_id').Distance.max().reset_index(name='u37')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 优惠券类型
    discount_types = [
        '0.2', '0.5', '0.6', '0.7', '0.75', '0.8', '0.85', '0.9', '0.95', '30:20', '50:30', '10:5',
        '20:10', '100:50', '200:100', '50:20', '30:10', '150:50', '100:30', '20:5', '200:50', '5:1',
        '50:10', '100:20', '150:30', '30:5', '300:50', '200:30', '150:20', '10:1', '50:5', '100:10',
        '200:20', '300:30', '150:10', '300:20', '500:30', '20:1', '100:5', '200:10', '30:1', '150:5',
        '300:10', '200:5', '50:1', '100:1',
    ]
    X['discount_type'] = -1
    for k, v in enumerate(discount_types):
        X.loc[X.Discount_rate == v, 'discount_type'] = k

    # 不同优惠券领取次数
    temp = offline.groupby(['User_id', 'Discount_rate']).size().reset_index(name='u41')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Discount_rate'])

    # 不同优惠券使用次数
    temp = coupon_consume.groupby(['User_id', 'Discount_rate']).size().reset_index(name='u42')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Discount_rate'])

    # 不同优惠券不使用次数
    temp = coupon_no_consume.groupby(['User_id', 'Discount_rate']).size().reset_index(name='u43')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Discount_rate'])

    # 不同打折优惠券使用率
    X['u44'] = X.u42 / X.u41

    # 满减类型优惠券领取次数
    temp = offline[offline.Discount_rate.str.contains(':') == True]
    temp = temp.groupby('User_id').size().reset_index(name='u48')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 打折类型优惠券领取次数
    temp = offline[offline.Discount_rate.str.contains('\.') == True]
    temp = temp.groupby('User_id').size().reset_index(name='u49')
    X = pd.merge(X, temp, how='left', on='User_id')

    '''offline merchant features'''

    # 商家销售次数
    temp = offline[offline.Date.notnull()].groupby('Merchant_id').size().reset_index(name='m0')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家优惠券核销次数
    temp = coupon_consume.groupby('Merchant_id').size().reset_index(name='m1')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家普通销售次数
    X['m2'] = X.m0.fillna(0) - X.m1.fillna(0)

    # 商家优惠券被领取次数
    temp = date_received_notnull.groupby('Merchant_id').size().reset_index(name='m3')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家优惠券核销率
    X['m4'] = X.m1 / X.m3

    # 商家优惠券不核销次数
    temp = coupon_no_consume.groupby('Merchant_id').size().reset_index(name='m7')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家在数据集中出现的次数
    temp = offline.groupby('Merchant_id').size().reset_index(name='m16')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商户当天优惠券领取次数
    temp = X[X.Date_received.notnull()]
    temp = temp.groupby(['Merchant_id', 'Date_received']).size().reset_index(name='m5')
    X = pd.merge(X, temp, how='left', on=['Merchant_id', 'Date_received'])

    # 商家当天优惠券领取人数
    temp = X[X.Date_received.notnull()]
    temp = temp.groupby(['User_id', 'Merchant_id', 'Date_received']).size()
    temp = temp.groupby(['Merchant_id', 'Date_received']).size().reset_index(name='m6')
    X = pd.merge(X, temp, how='left', on=['Merchant_id', 'Date_received'])

    # 商家核销优惠券的平均折率
    temp = coupon_consume.groupby('Merchant_id').discount.mean().reset_index(name='m8')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家优惠券核销的最小消费折率
    temp = coupon_consume.groupby('Merchant_id').discount.max().reset_index(name='m9')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家优惠券核销的最大消费折率
    temp = coupon_consume.groupby('Merchant_id').discount.min().reset_index(name='m10')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家优惠券核销不同的用户数量
    temp = coupon_consume.groupby(['Merchant_id', 'User_id']).size()
    temp = temp.groupby('Merchant_id').size().reset_index(name='m11')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家优惠券领取不同的用户数量
    temp = date_received_notnull.groupby(['Merchant_id', 'User_id']).size()
    temp = temp.groupby('Merchant_id').size().reset_index(name='m12')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 核销商家优惠券的不同用户数量其占领取不同的用户比重
    X['m13'] = X.m11 / X.m12

    # 商家优惠券平均每个用户核销多少张
    X['m14'] = X.m1 / X.m12

    # 商家被核销过的不同优惠券数量
    temp = coupon_consume.groupby(['Merchant_id', 'Coupon_id']).size()
    temp = temp.groupby('Merchant_id').size().reset_index(name='m15')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家领取过的不同优惠券数量的比重
    temp = date_received_notnull.groupby(['Merchant_id', 'Coupon_id']).size()
    temp = temp.groupby('Merchant_id').count().reset_index(name='m18')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家被核销过的不同优惠券数量占所有领取过的不同优惠券数量的比重
    X['m19'] = X.m15 / X.m18

    # 商家被核销优惠券的平均时间
    temp = pd.merge(coupon_consume, coupon_consume.groupby('Merchant_id').Date.max().reset_index(name='_max'))
    temp = pd.merge(temp, temp.groupby('Merchant_id').Date.min().reset_index(name='_min'))
    temp = pd.merge(temp, temp.groupby('Merchant_id').size().reset_index(name='_len'))
    temp['m20'] = (temp._max - temp._min).dt.days / (temp._len - 1)
    temp.drop_duplicates('Merchant_id', inplace=True)
    X = pd.merge(X, temp[['Merchant_id', 'm20']], how='left', on='Merchant_id')

    # 商家被核销优惠券中的用户平均距离
    temp = coupon_consume[coupon_consume.Distance.notnull()].groupby('Merchant_id').Distance
    temp = pd.merge(temp.count().reset_index(name='x'), temp.sum().reset_index(name='y'), on='Merchant_id')
    temp['m21'] = temp.y / temp.x
    temp = temp[['Merchant_id', 'm21']]
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家被核销优惠券中的用户最小距离
    temp = coupon_consume[coupon_consume.Distance.notnull()]
    temp = temp.groupby('Merchant_id').Distance.min().reset_index(name='m22')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家被核销优惠券中的用户最大距离
    temp = coupon_consume[coupon_consume.Distance.notnull()]
    temp = temp.groupby('Merchant_id').Distance.max().reset_index(name='m23')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家普通消费用户的平均距离
    # temp = offline[pd.isnull(offline.Date_received) & pd.notnull(offline.Date) & pd.notnull(offline.Distance)]
    # temp = temp.groupby('Merchant_id').Distance
    # temp = pd.merge(temp.count().reset_index(name='x'), temp.sum().reset_index(name='y'), on='Merchant_id')
    # temp['m24'] = temp.y / temp.x
    # temp = temp[['Merchant_id', 'm24']]
    # X = pd.merge(X, temp, how='left', on='Merchant_id')

    """offline coupon features"""

    # 每种优惠券领取次数
    temp = coupon_id_notnull.groupby('Coupon_id').size().reset_index(name='c1')
    X = pd.merge(X, temp, how='left', on='Coupon_id')

    # 每种优惠券使用次数
    temp = coupon_consume.groupby('Coupon_id').size().reset_index(name='c2')
    X = pd.merge(X, temp, how='left', on='Coupon_id')

    # 优惠券使用率
    X['c3'] = X.c2 / X.c1

    # 优惠券没有使用次数
    X['c4'] = X.c1 - X.c2

    # 优惠券当天领取次数
    temp = X.groupby(['Coupon_id', 'Date_received']).size().reset_index(name='c5')
    X = pd.merge(X, temp, how='left', on=['Coupon_id', 'Date_received'])

    # 优惠券类型(直接优惠为0, 满减为1)
    X['c6'] = 0
    X.loc[X.Discount_rate.str.contains(':') == True, 'c6'] = 1

    # 不同打折优惠券领取次数
    temp = offline.groupby('Discount_rate').size().reset_index(name='c8')
    X = pd.merge(X, temp, how='left', on='Discount_rate')

    # 不同打折优惠券使用次数
    temp = coupon_consume.groupby('Discount_rate').size().reset_index(name='c9')
    X = pd.merge(X, temp, how='left', on='Discount_rate')

    # 不同打折优惠券不使用次数
    temp = coupon_no_consume.groupby('Discount_rate').size().reset_index(name='c10')
    X = pd.merge(X, temp, how='left', on='Discount_rate')

    # 不同打折优惠券使用率
    X['c11'] = X.c9 / X.c8

    # 每种优惠券核销的平均时间
    temp = pd.merge(coupon_consume, coupon_consume.groupby('Coupon_id').Date.max().reset_index(name='_max'))
    temp = pd.merge(temp, temp.groupby('Coupon_id').Date.min().reset_index(name='_min'))
    temp = pd.merge(temp, temp.groupby('Coupon_id').size().reset_index(name='_len'))
    temp['c12'] = (temp._max - temp._min).dt.days / (temp._len - 1)
    temp.drop_duplicates('Coupon_id', inplace=True)
    X = pd.merge(X, temp[['Coupon_id', 'c12']], how='left', on='Coupon_id')

    '''user merchant feature'''

    # 用户领取商家的优惠券次数
    temp = coupon_id_notnull
    temp = temp.groupby(['User_id', 'Merchant_id']).size().reset_index(name='um1')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id'])

    # 用户领取商家的优惠券后不核销次数
    temp = coupon_no_consume.groupby(['User_id', 'Merchant_id']).size().reset_index(name='um2')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id'])

    # 用户领取商家的优惠券后核销次数
    temp = coupon_consume.groupby(['User_id', 'Merchant_id']).size().reset_index(name='um3')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id'])

    # 用户领取商家的优惠券后核销率
    X['um4'] = X.um3 / X.um1

    # 用户对每个商家的不核销次数占用户总的不核销次数的比重
    temp = coupon_no_consume.groupby('User_id').size().reset_index(name='temp')
    X = pd.merge(X, temp, how='left', on='User_id')
    X['um5'] = X.um2 / X.temp
    X.drop(columns='temp', inplace=True)

    # 用户在商店总共消费过几次
    temp = offline[offline.Date.notnull()].groupby(['User_id', 'Merchant_id']).size().reset_index(name='um6')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id'])

    # 用户在商店普通消费次数
    temp = date_received_isnull_date_notnull
    temp = temp.groupby(['User_id', 'Merchant_id']).size().reset_index(name='um7')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id'])

    # 用户当天在每个商家领取的优惠券次数
    temp = date_received_notnull
    temp = temp.groupby(['User_id', 'Merchant_id', 'Date_received']).size().reset_index(name='um8')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id', 'Date_received'])

    # 用户领取优惠券不同商家数量
    temp = coupon_id_notnull
    temp = temp.groupby(['User_id', 'Merchant_id']).size().reset_index()
    temp = temp.groupby('User_id').size().reset_index(name='um9')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户核销优惠券不同商家数量
    temp = coupon_consume.groupby(['User_id', 'Merchant_id']).size()
    temp = temp.groupby('User_id').size().reset_index(name='um10')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户核销过优惠券的不同商家数量占所有不同商家的比重
    X['um11'] = X.um10 / X.um9

    # 用户平均核销每个商家多少张优惠券
    X['um12'] = X.u2 / X.um9

    '''other feature'''

    # 用户领取优惠券次数
    temp = X.groupby('User_id').size().reset_index(name='o1')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户不同优惠券领取次数
    temp = X.groupby(['User_id', 'Coupon_id']).size().reset_index(name='o2')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Coupon_id'])

    # 用户此次之后/前领取的优惠券次数
    X['o3'] = 1
    X['o3'] = X.sort_values(by=['User_id', 'Date_received']).groupby('User_id').o3.cumsum() - 1
    X['o4'] = 1
    X['o4'] = X.sort_values(by=['User_id', 'Date_received'], ascending=False).groupby('User_id').o4.cumsum() - 1

    # 用户此次之后/前领取的每种优惠券次数
    X['o5'] = 1
    temp = X.sort_values(by=['User_id', 'Coupon_id', 'Date_received'])
    X['o5'] = temp.groupby('User_id').o5.cumsum() - 1
    X['o6'] = 1
    temp = X.sort_values(by=['User_id', 'Coupon_id', 'Date_received'], ascending=False)
    X['o6'] = temp.groupby('User_id').o6.cumsum() - 1

    # 用户领取优惠券平均时间间隔
    temp = pd.merge(X, X.groupby('User_id').Date_received.max().reset_index(name='_max'))
    temp = pd.merge(temp, temp.groupby('User_id').Date_received.min().reset_index(name='_min'))
    temp = pd.merge(temp, temp.groupby('User_id').size().reset_index(name='_len'))
    temp['o7'] = (temp._max - temp._min).dt.days / (temp._len - 1)
    temp.drop_duplicates('User_id', inplace=True)
    X = pd.merge(X, temp[['User_id', 'o7']], how='left', on='User_id')

    # 用户领取不同商家的优惠券次数
    temp = X.groupby(['User_id', 'Merchant_id']).size().reset_index(name='o8')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id'])

    # 用户领取的不同商家数
    temp = X.groupby(['User_id', 'Merchant_id']).size()
    temp = temp.groupby('User_id').size().reset_index(name='o9')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户当天领取的优惠券次数
    temp = X.groupby(['User_id', 'Date_received']).size().reset_index(name='o10')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Date_received'])

    # 用户当天不同优惠券领取次数
    temp = X.groupby(['User_id', 'Coupon_id', 'Date_received']).size().reset_index(name='o11')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Coupon_id', 'Date_received'])

    # 用户领取优惠券类别数
    temp = X.groupby(['User_id', 'Coupon_id']).size()
    temp = temp.groupby('User_id').size().reset_index(name='o12')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 商家被领取的优惠券次数
    temp = X.groupby('Merchant_id').size().reset_index(name='o13')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家优惠券的种类数
    temp = X.groupby(['Merchant_id', 'Coupon_id']).size().reset_index(name='o14')
    X = pd.merge(X, temp, how='left', on=['Merchant_id', 'Coupon_id'])

    # 商家被领取优惠券不同用户数
    temp = X.groupby(['Merchant_id', 'User_id']).size()
    temp = temp.groupby('Merchant_id').size().reset_index(name='o15')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家优惠券所有种类数
    temp = X.groupby(['Merchant_id', 'Coupon_id']).size()
    temp = temp.groupby('Merchant_id').size().reset_index(name='o16')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 用户领取优惠券的时间间隔
    temp = X.sort_values(by=['User_id', 'Date_received']).groupby('User_id')
    X['o17'] = temp.Date_received.diff().dt.days
    X['o18'] = temp.Date_received.diff(-1).dt.days.abs()

    print(len(X), len(X.columns))

    return X


def get_online_features(X, online):
    # temp = online[online.Coupon_id == online.Coupon_id]
    # coupon_consume = temp[temp.Date == temp.Date]
    # coupon_no_consume = temp[temp.Date != temp.Date]

    # 用户操作次数
    temp = online.groupby('User_id').size().reset_index(name='on_u1')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户点击次数
    temp = online[online.Action == 0].groupby('User_id').size().reset_index(name='on_u2')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户点击率
    X['on_u3'] = X.on_u2 / X.on_u1

    # 用户购买次数
    temp = online[online.Action == 1].groupby('User_id').size().reset_index(name='on_u4')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户线上购买率
    X['on_u5'] = X.on_u4 / X.on_u1

    # 用户领取优惠券次数
    temp = online[online.Date_received.notnull()].groupby('User_id').size().reset_index(name='on_u6')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户优惠券领取率
    X['on_u7'] = X.on_u6 / X.on_u1

    # 用户优惠券消费次数
    temp = online[online.Date_received.notnull() & online.Date.notnull()]
    temp = temp.groupby('User_id').size().reset_index(name='on_u9')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户优惠券不消费次数
    temp = online[online.Date_received.notnull() & online.Date.isnull()]
    temp = temp.groupby('User_id').size().reset_index(name='on_u8')
    X = pd.merge(X, temp, how='left', on='User_id')

    # 用户优惠券核销率
    X['on_u10'] = X.on_u9 / X.on_u6

    # 用户线下不消费次数占线上线下总的不消费次数的比重
    X['on_u11'] = X.u3 / (X.on_u8.fillna(0) + X.u3.fillna(0))

    # 用户线下的优惠券核销次数占线上线下总的优惠券核销次数的比重
    X['on_u12'] = X.u2 / (X.on_u9.fillna(0) + X.u2.fillna(0))

    # 用户线下领取的记录数量占总的记录数量的比重
    X['on_u13'] = X.u1 / (X.on_u6.fillna(0) + X.u1.fillna(0))

    # # 消费优惠券的平均折率
    # temp = coupon_consume.groupby('User_id').discount.mean().reset_index(name='ou14')
    # X = pd.merge(X, temp, how='left', on='User_id')
    #
    # # 用户核销优惠券的最低消费折率
    # temp = coupon_consume.groupby('User_id').discount.min().reset_index(name='ou15')
    # X = pd.merge(X, temp, how='left', on='User_id')
    #
    # # 用户核销优惠券的最高消费折率
    # temp = coupon_consume.groupby('User_id').discount.max().reset_index(name='ou16')
    # X = pd.merge(X, temp, how='left', on='User_id')
    #
    # # 不同打折优惠券领取次数
    # temp = online.groupby('Discount_rate').size().reset_index(name='oc1')
    # X = pd.merge(X, temp, how='left', on='Discount_rate')
    #
    # # 不同打折优惠券使用次数
    # temp = coupon_consume.groupby('Discount_rate').size().reset_index(name='oc2')
    # X = pd.merge(X, temp, how='left', on='Discount_rate')
    #
    # # 不同打折优惠券不使用次数
    # temp = coupon_no_consume.groupby('Discount_rate').size().reset_index(name='oc3')
    # X = pd.merge(X, temp, how='left', on='Discount_rate')
    #
    # # 不同打折优惠券使用率
    # X['oc4'] = X.oc2 / X.oc1

    print(len(X), len(X.columns))
    print('----------')

    return X


In [97]:
train = get_offline_features(train, train_offline)
train = get_online_features(train, train_online)
test = get_offline_features(test, train_offline)
test = get_online_features(test, train_online)

113640 6
113640 108
113640 121
----------


In [105]:
print('test_ dataset has {} rows and {} columns.'.format(test.shape[0],test.shape[1]))
print('train_ dataset has {} rows and {} columns.'.format(train.shape[0],train.shape[1]))

test_ dataset has 113640 rows and 122 columns.
train_ dataset has 1754884 rows and 124 columns.


In [107]:
test_features = test.columns
trainfeatures = train.columns

for feature in trainfeatures:
  if feature not in test_features:
    print(feature)

Date
label


In [108]:
train.to_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/train2.csv', index=0)
test.to_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/test2.csv', index=0)